In [1]:
from google.cloud import bigquery
bq = bigquery.Client()

In [2]:
q = bq.query(
"""
select count(*) as num_rows 
from bigquery-public-data.geo_us_boundaries.counties
""")
q.to_dataframe()

,num_rows
0,3233


In [3]:
#Q1
q1 = bq.query(
"""
select geo_id
from bigquery-public-data.geo_us_boundaries.counties
where county_name = 'Dane'
""")
q1 = q1.to_dataframe()
for name in q1["geo_id"]:
    id = name
id

'55025'

In [4]:
#q2

q2 = bq.query(
"""
select state_fips_code, count(*) as num_counties
from bigquery-public-data.geo_us_boundaries.counties
GROUP BY state_fips_code
order by num_counties DESC
limit 5
""")
q2 = q2.to_dataframe()
q2_dict = {}
for name,num in zip(q2["state_fips_code"],q2["num_counties"]):
    q2_dict[name] = num
q2_dict

{'48': 254, '13': 159, '51': 133, '21': 120, '29': 115}

In [5]:
no_cache = bigquery.QueryJobConfig(use_query_cache=False)

In [6]:
#q3
tb = 1024 **4
q1 = bq.query(
"""
select geo_id
from bigquery-public-data.geo_us_boundaries.counties
where county_name = 'Dane'
""", job_config = no_cache)
q2 = bq.query(
"""
select state_fips_code, count(*) as num_counties
from bigquery-public-data.geo_us_boundaries.counties
GROUP BY state_fips_code
order by num_counties DESC
limit 5
""",job_config = no_cache)


q1_cost = q1.total_bytes_billed / tb * 6.25
q2_cost = q2.total_bytes_billed / tb * 6.25

{"q1": q1_cost, "q2": q2_cost}

{'q1': 5.9604644775390625e-05, 'q2': 5.9604644775390625e-05}

In [7]:
ds = bigquery.Dataset("operating-pact-398418.p8")
bq.create_dataset(ds,exists_ok=True)

Dataset(DatasetReference('operating-pact-398418', 'p8'))

In [8]:
config = bigquery.LoadJobConfig(source_format="PARQUET", write_disposition="WRITE_TRUNCATE")
source = "gs://cs544_p8_lol/hdma-wi-2021.parquet"
job = bq.load_table_from_uri(source, "operating-pact-398418.p8.hdma", job_config=config)
job.result()

LoadJob<project=operating-pact-398418, location=US, id=13d263e8-2ebe-4f4f-b4fd-69675a5854ea>

In [9]:
#q4
[ds.dataset_id for ds in bq.list_datasets("operating-pact-398418")]

['p8']

In [10]:
#q5
q5 = bq.query(
"""
SELECT counties.county_name, COUNT(*) as count
FROM operating-pact-398418.p8.hdma as hdma
JOIN bigquery-public-data.geo_us_boundaries.counties as counties
ON hdma.county_code = counties.county_fips_code
GROUP BY counties.county_name
ORDER BY count DESC
LIMIT 10;

""")
q5_dict = {row['county_name']: row['count'] for row in q5.to_dataframe().to_dict(orient='records')}
q5_dict

{'Milwaukee': 46570,
 'Dane': 38557,
 'Waukesha': 34159,
 'Brown': 15615,
 'Racine': 13007,
 'Outagamie': 11523,
 'Kenosha': 10744,
 'Washington': 10726,
 'Rock': 9834,
 'Winnebago': 9310}

In [11]:
url = "https://docs.google.com/spreadsheets/d/11UeIBqQylAyNUBsIO54p6WiYJWHayQMfHDbUWq1jGco/"

external_config = bigquery.ExternalConfig("GOOGLE_SHEETS")
external_config.source_uris = ["https://docs.google.com/spreadsheets/d/11UeIBqQylAyNUBsIO54p6WiYJWHayQMfHDbUWq1jGco/"]
external_config.options.skip_leading_rows = 1
external_config.autodetect = True

table = bigquery.Table("operating-pact-398418.p8.applications")
table.external_data_configuration = external_config

table = bq.create_table(table, exists_ok=True)

In [12]:
#q6
q6 = bq.query(
"""
SELECT count(*)
FROM operating-pact-398418.p8.applications
WHERE income = 100000
""")
q6.to_dataframe().iloc[0, 0]

37

In [13]:
#q7
q7 = bq.query(
"""
SELECT counties.county_name as county_name, COUNT(*) as county_count
FROM bigquery-public-data.geo_us_boundaries.counties AS counties
JOIN operating-pact-398418.p8.applications AS applications
ON ST_CONTAINS(counties.county_geom, ST_GEOGPOINT(applications.longitude, applications.latitude))
WHERE counties.state_fips_code = '55'
GROUP BY counties.county_name
"""
)
q7 = q7.to_dataframe()
q7_dict = {}
for name,num in zip(q7["county_name"],q7["county_count"]):
    q7_dict[name] = num
q7_dict

{'Dane': 67,
 'Door': 19,
 'Oneida': 3,
 'Jefferson': 2,
 'Brown': 6,
 'Chippewa': 1,
 'Kenosha': 1,
 'Monroe': 2,
 'Winnebago': 3,
 'Eau Claire': 1,
 'Bayfield': 5,
 'Milwaukee': 4,
 'Sheboygan': 5,
 'Walworth': 4,
 'Ashland': 2,
 'Kewaunee': 3,
 'Crawford': 1,
 'Columbia': 4,
 'Marinette': 2,
 'Green Lake': 2,
 'Douglas': 1,
 'Wood': 2,
 'Manitowoc': 1,
 'Iowa': 1,
 'Barron': 2,
 'Outagamie': 1,
 'Ozaukee': 1,
 'Sauk': 2,
 'St. Croix': 1,
 'Adams': 1,
 'Dodge': 1,
 'Marathon': 1,
 'Polk': 1}

In [14]:
%load_ext google.cloud.bigquery

In [15]:
%%bigquery
CREATE OR REPLACE MODEL `operating-pact-398418.p8.lr`
OPTIONS(model_type="LINEAR_REG", INPUT_LABEL_COLS=["loan_amount"])

AS

SELECT income, loan_term, loan_amount
FROM p8.hdma

Query is running:   0%|          |

""


In [16]:
import time
client = bigquery.Client(project='operating-pact-398418')
model_name = 'lr'
def model_exists():
    models = list(client.list_models('p8'))
    model_ids = [model.model_id for model in models]
    print(model_ids)
    return model_name in model_ids

while True:
    if model_exists():
        break
    # debug statement
    print("Waiting for model")
    time.sleep(5)

['lr']


In [17]:
%%bigquery df
SELECT *
FROM ML.EVALUATE(MODEL `operating-pact-398418.p8.lr`, (
    SELECT income, loan_term, loan_amount
    FROM p8.hdma
))

Query is running:   0%|          |

Downloading:   0%|          |

In [18]:
#q8
df.at[0, "r2_score"]

0.291654122880196

In [19]:
%%bigquery df2
SELECT *
FROM ML.WEIGHTS(MODEL `operating-pact-398418.p8.lr`)

Query is running:   0%|          |

Downloading:   0%|          |

In [20]:
#q9
df2.at[0, "weight"]

0.8057773037176723

In [21]:
%%bigquery df3
SELECT COUNTIF(predicted_loan_amount > loan_amount) / COUNT(*) AS ratio_greater_than_predicted
FROM ML.PREDICT(MODEL `operating-pact-398418.p8.lr`, (
    SELECT income, 360 as loan_term, loan_amount
    FROM p8.applications
))

Query is running:   0%|          |

Downloading:   0%|          |

In [22]:
#q10
df3.at[0, "ratio_greater_than_predicted"]

0.3937007874015748